In [2]:
# Initial imports
import pandas as pd
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
# from sklearn.metrics import score
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingRegressor
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt

# Needed for decision tree visualization
import pydotplus
from IPython.display import Image

In [3]:
df = pd.read_csv('etc.csv',parse_dates=True, index_col=('date'))
df.dropna(inplace=True)
df.rename(columns={'date':'Date','PriceUSD':'Close'}, inplace=True)
futures = df.tail()

df1 = df[['FeeMedUSD','TxTfrValMedUSD','Close','IssTotUSD','VtyDayRet30d']]
futures1 = df1.tail()

df2 = df[['FeeMedUSD','TxTfrValMedUSD','Close','IssTotUSD','VtyDayRet30d','TxTfrValMeanNtv']]
futures2 = df2.tail()


In [4]:
# df.Close.head()

In [5]:
# df = df[:-3]
df.Close.tail()

date
2019-11-06    5.077399
2019-11-07    5.225557
2019-11-08    4.984974
2019-11-09    4.956965
2019-11-10    5.038860
Name: Close, dtype: float64

In [6]:
#Model0
X=df[:-1].copy()
X.head()
y = df["Close"][1:].values.reshape(-1, 1)
y[:5]
#Model1
X1=df1[:-1].copy()
X1=X1[['FeeMedUSD','TxTfrValMedUSD','Close','IssTotUSD','VtyDayRet30d']]
y1 = df1["Close"][1:].values.reshape(-1, 1)
y1[:5]
#Model2
X2=df2[:-1].copy()
X2=X2[['FeeMedUSD','TxTfrValMedUSD','IssTotUSD','VtyDayRet30d','TxTfrValMeanNtv']]
y2 = df2["Close"][1:].values.reshape(-1, 1)
y2[:5]

array([[14.64347978],
       [14.4889509 ],
       [13.91851153],
       [14.28201887],
       [13.77123374]])

In [7]:
X.tail()

,AdrActCnt,BlkCnt,BlkSizeByte,BlkSizeMeanByte,CapMrktCurUSD,DiffMean,FeeMeanNtv,FeeMeanUSD,FeeMedNtv,FeeMedUSD,...,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,TxTfrValNtv,TxTfrValUSD,VtyDayRet180d,VtyDayRet30d,VtyDayRet60d,High,Low
date,,,,,,,,,,,,,,,,,,,,,
2019-11-05,450544,6560,10060327,1533.586433,577760237.2,1.230000e+14,0.000179,0.000898,0.000021,0.000105,...,120.393431,0.999342,5.018602,8.351223e+05,4193905.185,0.048304,0.036883,0.039343,5.09,4.90
2019-11-06,461538,6581,10061580,1528.883148,584278644.3,1.250000e+14,0.000172,0.000876,0.000021,0.000107,...,134.941052,0.999569,5.075210,9.200360e+05,4671389.328,0.048303,0.036105,0.039394,5.17,5.02
2019-11-07,472837,6652,10245661,1540.237673,601467267.3,1.280000e+14,0.000183,0.000957,0.000021,0.000110,...,234.100925,0.999707,5.224026,1.557535e+06,8138986.874,0.047595,0.036280,0.039627,5.30,5.07
2019-11-08,457844,6524,10080229,1545.099479,573906269.3,1.290000e+14,0.000181,0.000904,0.000021,0.000105,...,203.683909,0.999356,4.981765,1.413864e+06,7048074.319,0.047400,0.033169,0.040016,5.29,4.85
2019-11-09,448899,6520,9745342,1494.684356,570811384.3,1.260000e+14,0.000162,0.000802,0.000021,0.000104,...,104.631391,0.999052,4.952268,7.036547e+05,3487992.064,0.047304,0.031993,0.039972,5.05,4.95


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, random_state=78)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state=78)

In [9]:
# Scale the training data
scaler = StandardScaler()
scaler1=StandardScaler()
scaler2=StandardScaler()
#Model0
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
#model1
X_scaler1 = scaler1.fit(X_train1)
X_train_scaled1 = X_scaler1.transform(X_train1)
X_test_scaled1 = X_scaler1.transform(X_test1)
#model2
X_scaler2 = scaler2.fit(X_train2)
X_train_scaled2 = X_scaler2.transform(X_train2)
X_test_scaled2 = X_scaler2.transform(X_test2)

In [10]:
# Create GradientBoostingClassifier model
#Changed from .75 to .25 11/8
#on 11/8 .75 gave an accuracy score of .929
#model0
model = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=.25,
    max_features=5,
    max_depth=3,
    random_state=0)

# Fit the model
model.fit(X_train_scaled,y_train.ravel())

# Score the model
print("Accuracy score (training): {0:.3f}".format(
    model.score(
        X_train_scaled,
        y_train)))
print("Accuracy score (validation): {0:.3f}".format(
    model.score(
        X_test_scaled,
        y_test)))

Accuracy score (training): 1.000
Accuracy score (validation): 0.974


In [11]:
#model1 - [['FeeMedUSD','TxTfrValMedUSD','Close','IssTotUSD','VtyDayRet30d']]
# Create GradientBoostingClassifier model
#Changed from 1 to .05 11/8
#on 11/8 1 gave an accuracy score of .911
model1 = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=.05,
    max_features=5,
    max_depth=3,
    random_state=0)

# Fit the model
model1.fit(X_train_scaled1,y_train1.ravel())

# Score the model
print("Accuracy score (training): {0:.3f}".format(
    model1.score(
        X_train_scaled1,
        y_train1)))
print("Accuracy score (validation): {0:.3f}".format(
    model1.score(
        X_test_scaled1,
        y_test1)))

Accuracy score (training): 0.998
Accuracy score (validation): 0.978


In [12]:
#model2 - [['FeeMedUSD','TxTfrValMedUSD','IssTotUSD','VtyDayRet30d','TxTfrValMeanNtv']]
#Changed from .1 to .05 11/8
#on 11/8 .1 gave an accuracy score of .938
model2 = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=.05,
    max_features=5,
    max_depth=3,
    random_state=0)

# Fit the model
model2.fit(X_train_scaled2,y_train2.ravel())

# Score the model
print("Accuracy score (training): {0:.3f}".format(
    model2.score(
        X_train_scaled2,
        y_train2)))
print("Accuracy score (validation): {0:.3f}".format(
    model2.score(
        X_test_scaled2,
        y_test2)))

Accuracy score (training): 0.998
Accuracy score (validation): 0.967


In [13]:
# Make predictions
#model0
predictions = model.predict(X_test_scaled)
y_test = y_test.ravel()
actual = pd.Series(y_test)

#model1
predictions1 = model1.predict(X_test_scaled1)
y_test1 = y_test1.ravel()
actual1 = pd.Series(y_test1)
#model2
predictions2 = model2.predict(X_test_scaled2)
y_test2 = y_test2.ravel()
actual2 = pd.Series(y_test2)

In [14]:
pd.DataFrame({'Model0':predictions,'Model1':predictions1,'Model2':predictions2,'Actual': actual}).head()

,Model0,Model1,Model2,Actual
0,35.932554,33.875286,34.906849,35.242843
1,4.604531,4.542062,4.570346,4.709501
2,13.942976,13.832268,13.749405,14.565106
3,8.811563,8.507541,8.559469,8.815582
4,5.263298,5.499843,5.414673,6.044361


In [15]:
X.tail().shape


(5, 39)

In [16]:
X_future = X_scaler.transform(X.tail(5))
predictions = model.predict(X_future)

X_future1 = X_scaler1.transform(X1.tail(5))
predictions1 = model1.predict(X_future1)

X_future2 = X_scaler2.transform(X2.tail(5))
predictions2 = model2.predict(X_future2)

In [17]:
actual=futures['Close'].ravel()
dates=futures.reset_index()
dates=dates['date'].tail(5).ravel()
#Model1
actual1=futures1['Close'].ravel()
dates1=futures1.reset_index()
dates1=dates1['date'].tail(5).ravel()
#Model2
actual2=futures2['Close'].ravel()
dates2=futures2.reset_index()
dates2=dates2['date'].tail(5).ravel()

In [18]:
test_final=pd.DataFrame({'Model0':predictions,'Model1':predictions1,'Model2':predictions2,'Actual':actual, 'Date':dates})

In [19]:
df.Close.tail()

date
2019-11-06    5.077399
2019-11-07    5.225557
2019-11-08    4.984974
2019-11-09    4.956965
2019-11-10    5.038860
Name: Close, dtype: float64

In [24]:
#Model0
real_future = X_scaler.transform(df.tail(1))
predictions = model.predict(real_future)
#Model1
real_future1 = X_scaler1.transform(df1.tail(1))
predictions1 = model1.predict(real_future1)
#Model2
real_future2 = X_scaler2.transform(df2.tail(1).drop('Close',axis=1))
predictions2 = model2.predict(real_future2)

Based on my predictions today, (Nov 7th)- I should sell my stock tomorrow (Nov 8th)


In [27]:
datelist = pd.date_range((pd.datetime.today()), periods=1).tolist()

# datelist = pd.date_range((pd.datetime.today()+timedelta(days=-3)), periods=5).tolist()
future_dates=[]
for date in datelist:
    future_dates.append(date.strftime("%m-%d-%Y"))

In [28]:
final=pd.DataFrame({'Date':future_dates,'Model0':predictions,'Model1':predictions1,'Model2':predictions2})
final['Avg']=(final.Model0+final.Model1+final.Model2) / 3

In [29]:
final

,Date,Model0,Model1,Model2,Avg
0,11-11-2019,4.994954,5.074858,5.324993,5.131602


In [23]:
# final.to_csv('11_11.csv')


In [24]:
final=pd.read_csv('final.csv',parse_dates=True, index_col=('Date')).drop('Unnamed: 0', axis=1)
final['Actual']=df.Close.tail(8)

In [25]:
final_M0=final[['M0_11-8','M0_11-9','M0_11-10','M0_11-11','Actual']]
# final_M0['Actual']=df.Close.tail(8)


In [26]:
final_M0

,M0_11-8,M0_11-9,M0_11-10,M0_11-11,Actual
Date,,,,,
2019-11-08,5.180671,NaN,NaN,NaN,4.984974
2019-11-09,5.026740,5.036135,NaN,NaN,4.956965
2019-11-10,5.008105,5.190125,4.995127,NaN,5.038860
2019-11-11,5.613164,5.809133,5.826352,5.771785,NaN
2019-11-12,5.299263,5.795424,5.467567,5.580944,NaN
2019-11-13,NaN,5.393370,5.030278,5.125139,NaN
2019-11-14,NaN,NaN,5.157152,5.159809,NaN
2019-11-15,NaN,NaN,NaN,5.825111,NaN


In [27]:
cols = final.columns.tolist()
cols = cols[-1:] + cols[:-1]
final=final[cols]


In [28]:
final

,Actual,M0_11-8,M1_11-8,M2_11-8,Avg_11-8,M0_11-9,M1_11-9,M2_11-9,Avg_11-9,M0_11-10,M1_11-10,M2_11-10,Avg_11-10,M0_11-11,M1_11-11,M2_11-11,Avg_11-11
Date,,,,,,,,,,,,,,,,,
2019-11-08,4.984974,5.180671,4.893701,5.842542,5.305638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-09,4.956965,5.026740,5.012138,5.857872,5.298917,5.036135,4.893701,5.842542,5.257459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-10,5.038860,5.008105,4.972495,5.065607,5.015402,5.190125,5.012138,5.857872,5.353378,4.995127,4.893701,5.842542,5.243790,NaN,NaN,NaN,NaN
2019-11-11,NaN,5.613164,5.479370,5.256235,5.449590,5.809133,4.972495,5.065607,5.282412,5.826352,5.012138,5.857872,5.565454,5.771785,4.893701,5.842542,5.502676
2019-11-12,NaN,5.299263,5.480011,6.228255,5.669176,5.795424,5.479370,5.256235,5.510343,5.467567,4.972495,5.065607,5.168557,5.580944,5.012138,5.857872,5.483652
2019-11-13,NaN,NaN,NaN,NaN,NaN,5.393370,5.480011,6.228255,5.700546,5.030278,5.479370,5.256235,5.255295,5.125139,4.972495,5.065607,5.054414
2019-11-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.157152,5.480011,6.228255,5.621806,5.159809,5.479370,5.256235,5.298471
2019-11-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.825111,5.480011,6.228255,5.844459
